Basic Imports for chain runs

In [ ]:
import os
from functools import partial
import json

import geopandas as gpd
import matplotlib.pyplot as plt

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)
from gerrychain.metrics import efficiency_gap, mean_median
from gerrychain.proposals import recom
from gerrychain.updaters import cut_edges
from gerrychain.tree import recursive_tree_part

Create the underlying graph object from the json provided here https://github.com/vrdi/Your-State.
You will need to look up your states FIPS code as that is how everything is organized.

Most states will have voting and demographic information, however, no states have congressional districting assignments.
For the purposes of running begining chains we will generate a begining plan using recursive_tree_part from gerrychain.tree

In [ ]:
graph_path = "path to your json here"
graph = Graph.from_json(graph_path)

Next, set how many districts you want to generate, and set what the population is for your state.  To start you can just google what the estimated population count is for your state, and once you run chain you can calculate the actual population that exists on your graph.  This will be used to calculate the population deviation between districts.

After that, create your updaters, here I have put in population and cut_edges but you can do whatever you would like.

Finally, we will actually generate our starting plan and create the initial parition.  You will use recursive_tree_part for the plan.  The arguments in order are the graph object, the range of how many districts you set, the population target for each district, percentage of how far from ideal population we can be, the number of different choices of root to use before drawing a new spanning tree.  After that create your inital partiton using the Partition class by passing it the graph, the plan you just generated, and your updaters.

In [ ]:
num_dist  = 3
pop = 577737

updaters = {
    "population": updaters.Tally("TOTPOP", alias="population"),
    "cut_edges": cut_edges,
}

new_plan = recursive_tree_part(graph, range(num_dist), pop/num_dist, "TOTPOP", .05, 1)
initial_partition = Partition(graph, new_plan, updaters)

Now set how your walk will propse the next step.  For the purposes of this we will be using the recom step.  We will also set some basic constraints to give to the run.  You can add however many you want here, this is just a starting point.

In [ ]:
proposal = partial(recom, pop_col = "TOTPOP", pop_target = pop/num_dist, epsilon = 0.05, node_repeats = 3)

compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]), 2 * len(initial_partition["cut_edges"])
)

TIME TO MAKE THE CHAIN!

In [ ]:
chain = MarkovChain(
    proposal=proposal,
    constraints=[
        constraints.within_percent_of_ideal_population(initial_partition, 0.09),
        compactness_bound,  # single_flip_contiguous#no_more_discontiguous
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=10,
)

All that's left is to start walking!  This has been purposefully left without anything really useful.  This is where you will be looking at your metrics and evaluating what you find important!

In [ ]:
for step in chain:
    print(step["population"])